In [27]:
from __future__ import annotations

import asyncio
import time

from rich.console import Console
from printer import Printer
from agents import Runner, custom_span, gen_trace_id, trace

#! brew install graphicsmagick
#! pip install py-zerox
#! brew install poppler
from pyzerox import zerox
import docx
import json

In [2]:
import os
import sys

# Add current path to sys.path
sys.path.append(os.getcwd())
from printer import Printer

from helper_agents.resume_parser_agent import resume_parser_agent, ResumeItem
from helper_agents.jd_parser_agent import jd_parser_agent, web_scraper_tool, JDItem
from helper_agents.content_revise_agent import content_revise_agent
from helper_agents.expression_revise_agent import expression_revise_agent
from helper_agents.evaluation_agent import evaluation_agent, EvaluationResult

In [ ]:
class ResumeImprovementManager:
    def __init__(self):
        self.console = Console()
        self.printer = Printer(self.console)

    async def run(
        self, target_resume_file_path, jd_website, reference_resume_file_path: str
    ) -> None:
        # Generate a unique trace ID for this session
        trace_id = gen_trace_id()
        with trace("Resume Improvement trace", trace_id=trace_id):
            self.printer.update_item(
                "trace_id",
                f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}",
                is_done=True,
                hide_checkmark=True,
            )

            self.printer.update_item(
                "starting",
                "Starting research...",
                is_done=True,
                hide_checkmark=True,
            )
            # Read JD, and parse it
            jd_components = await self._get_jd_components(jd_website)
            # If jd_components is empty, handle it
            if not jd_components:
                self.printer.update_item("getting_jd", "No job description found.")
                return None
            elif jd_components.citizenship_requirements:
                self.printer.update_item("getting_jd", "Require citizenship.")
                return None
            # Get target resume and reference resume
            target_resume_component, reference_resume_component = (
                await self._convert_resume_files(
                    target_resume_file_path, reference_resume_file_path
                )
            )

            score = await self._evaluate_resume(
                target_resume_component, reference_resume_component, jd_components
            )

            revised_resume = target_resume_component

            # At most 3 iterations
            for i in range(3):
                if score.content_score < 8:
                    revised_resume = await self._revise_resume_content(
                        revised_resume, jd_components
                    )
                if score.expression_score < 8:
                    revised_resume = await self._revise_resume_expression(
                        revised_resume, reference_resume_component
                    )

            final_report = f"Final Revised Resume\n\n{revised_resume}"
            self.printer.update_item("final_report", final_report, is_done=True)

            self.printer.end()

        print("\n\n=====REPORT=====\n\n")
        print(f"Report: {revised_resume.markdown_report}")

    async def file_to_markdown(self, file_path, output_path):
        """Convert files to markdown. Local filepath and file URL supported"""
        # Ensure parent directory exists

        # If output file already exists, you may choose to overwrite or skip
        if os.path.isfile(output_path):
            return "The output file already exists. No conversion"
        if file_path.endswith(".md"):
            with open(file_path, "r") as f, open(output_path, "w") as out_f:
                out_f.write(f.read())
                return "Conversion done"
        if file_path.endswith(".docx"):
            doc = docx.Document(file_path)
            txt = "\n".join([p.text for p in doc.paragraphs])
            # save the txt to a md file
            with open(output_path, "w") as f:
                f.write(txt)
            return "Conversion done"

        model = "gpt-4o-mini"
        custom_system_prompt = """
            You are a resume parser. Extract the text from the PDF, 
            preserving headings, bullet points, and tables. 
            Return the output in Markdown format.
        """
        kwargs = {}
        # select_pages = None
        await zerox(
            file_path=file_path,
            model=model,
            output_dir=output_path,
            custom_system_prompt=custom_system_prompt,
            **kwargs,
        )
        return "Conversion done"

    async def _convert_resume_files(
        self, target_resume_file_path, reference_resume_file_path
    ):
        self.printer.update_item("converting", "Converting resume files...")
        # Convert both resume to markdown file if they are not
        converted_resume_target_path = "./converted_resume/target_resume.md"
        converted_resume_reference_path = "./converted_resume/reference_resume.md"
        outputmessage1 = await self.file_to_markdown(
            target_resume_file_path, converted_resume_target_path
        )
        self.printer.update_item("converting", outputmessage1)
        outputmessage2 = await self.file_to_markdown(
            reference_resume_file_path, converted_resume_reference_path
        )
        self.printer.update_item("converting", outputmessage2)

        with open(converted_resume_target_path, "r") as file:
            target_resume_content = file.read()
        with open(converted_resume_reference_path, "r") as file:
            reference_resume_content = file.read()

        # Get target resume components
        target_resume_components = await Runner.run(
            starting_agent=resume_parser_agent,
            input=f"Resume content: {target_resume_content}",
        )
        # Get reference resume components
        reference_resume_components = await Runner.run(
            starting_agent=resume_parser_agent,
            input=f"Resume content: {reference_resume_content}",
        )
        self.printer.mark_item_done("converting")

        # Return both resume components
        return target_resume_components.final_output_as(
            ResumeItem
        ), reference_resume_components.final_output_as(ResumeItem)

    async def _get_jd_components(self, jd_website: str) -> JDItem:
        self.printer.update_item("getting_jd", "Analyzing job description...")
        try:
            job_description = web_scraper_tool(jd_website)
        except Exception as e:
            self.printer.update_item("getting_jd", f"Error: {e}")
            # End the agent
            return None

        jd_components = await Runner.run(
            starting_agent=jd_parser_agent,
            input=f"Job description: {job_description}",
        )
        self.printer.mark_item_done("getting_jd")
        return jd_components.final_output_as(JDItem)

    async def _revise_resume_content(
        self,
        target_resume_components: ResumeItem,
        reference_resume_components: ResumeItem,
        jd_components: JDItem,
    ) -> ResumeItem:
        self.printer.update_item("revising_resume", "Revising resume content...")
        try:
            revised_resume = await Runner.run(
                starting_agent=content_revise_agent,
                input=json.dumps(
                    {
                        "target_resume_components": target_resume_components,
                        "jd_components": jd_components,
                    }
                ),
            )
            self.printer.mark_item_done("revising_resume")
            return revised_resume.final_output_as(ResumeItem)
        except Exception as e:
            self.printer.update_item("revising_resume", f"Error: {e}")
            return None

    async def _evaluate_resume(
        self,
        target_resume_components: ResumeItem,
        reference_resume_components: ResumeItem,
        jd_components: JDItem,
    ) -> EvaluationResult:
        self.printer.update_item("evaluating", "Evaluating resume...")
        result = await Runner.run(
            starting_agent=evaluation_agent,
            input=json.dumps(
                {
                    "target_resume_components": target_resume_components.dict(),
                    "reference_resume_components": reference_resume_components.dict(),
                    "job_description": jd_components.dict(),
                }
            ),
        )
        self.printer.mark_item_done("evaluating")
        return result.final_output_as(EvaluationResult)

    async def _revise_resume_expression(
        self,
        target_resume_components: ResumeItem,
        reference_resume_components: ResumeItem,
    ) -> ResumeItem:
        self.printer.update_item(
            "revising_expression", "Revising resume's expression..."
        )
        try:
            revised_resume = await Runner.run(
                starting_agent=content_revise_agent,
                input=json.dumps(
                    {
                        "target_resume_components": target_resume_components,
                        "reference_resume_components": reference_resume_components,
                    }
                ),
            )
            self.printer.mark_item_done("revising_expression")
            return revised_resume.final_output_as(ResumeItem)
        except Exception as e:
            self.printer.update_item("revising_expression", f"Error: {e}")
            return None

In [ ]:
import asyncio

# from manager import ResearchManager


await ResumeImprovementManager().run(
    "./target_resume.docx",
    "https://jobright.ai/jobs/info/68ace77ed627244576e49be6",
    "./converted_reference_resume/reference_resume.md",
)

In [29]:
example = ResumeImprovementManager()

In [5]:
jd_components = await example._get_jd_components(
    "https://jobright.ai/jobs/info/68ace77ed627244576e49be6"
)

In [6]:
target_resume_component, reference_resume_component = (
    await example._convert_resume_files(
        "./target_resume.docx", "./converted_resume/reference_resume.md"
    )
)

In [7]:
reference_resume_component

ResumeItem(summary='B-round startup veteran tech lead with eight years of cloud development experience across verticals, including healthcare, ML infra, and Edge AI agents. Skilled in multi-agent orchestration and extensive computing systems.', hard_skills=['C++', 'C#', 'Java', 'Python', 'SQL', 'Large Scale Distributed Systems', 'LLM Orchestration'], projects=[Project(title='Senior Software Development Engineer - Elastic Infra Platform', company='GLOBAL CLOUD INC.', project_name='Union Deployment', description=['Architected batch compute systems for next-gen SDP, helping CrowdStrike avoid another historic outage.', 'Achieved 99.99% update coverage for hybrid clouds serving RedRock, ClosedAI, Walmart, and BinaryDance.', 'Spearheaded cross-team efforts building data warehouses, ensuring global rollout visibility for leadership.', "Pioneered enhancing small LLMs' reasoning via RL self-play and MCTS to build an infra-rollout agent."], location='Seattle, WA'), Project(title='Software Develo

In [30]:
score = await example._evaluate_resume(
    target_resume_component, reference_resume_component, jd_components
)

In [31]:
score

EvaluationResult(content_score=8, expression_score=7)

In [36]:
revised_resume = target_resume_component

# At most 3 iterations
for i in range(3):
    if score.content_score < 8:
        revised_resume = example._revise_resume_content(
            target_resume_component, jd_components
        )
    if score.expression_score < 8:
        revised_resume = example._revise_resume_expression(
            target_resume_component, reference_resume_component
        )

final_report = f"Final Revised Resume\n\n{revised_resume}"

In [37]:
final_report

'Final Revised Resume\n\n<coroutine object ResumeImprovementManager._revise_resume_expression at 0x115444b40>'

In [39]:
json.dumps(revised_resume)

TypeError: Object of type coroutine is not JSON serializable